## Imports

In [1]:
import os
import json

parent_path = os.path.abspath("./../")
if os.path.basename(parent_path) == 'vdp':
    os.chdir(parent_path)

from glob import glob
import vdp
import re
import numpy as np


from vdp.vocabulary import FOSort, FOFunction
from vdp.fomodel import FOElement, FOModel
from vdp.exceptions import PartialInterpretationException




## Generate config.json files

In [2]:
# define the scene graph and the fo model generation configs.
sgconfig = {
"output_dir": "./../data/sg_processed",
"glove_path": "./../../checkpoints/glove",
"model_path": "./../../checkpoints/model",
"sg_tools_rel_path": "./tools/relation_test_net.py",
"sg_config_path": "./configs/e2e_relation_X_101_32_8_FPN_1x.yaml",
"cuda_device_port": 0,
'n_proc': 1
}

foconfig = {
    "input_dir": "./data/sg_processed",
    "output_dir": "./data/sg_ir",
    "rel_topk": 20, 
    "box_topk": 20
}

In [3]:
# write config files to memory.
vdp_puzzles =  glob("./data/raw/*")
os.makedirs("./config", exist_ok=True)
for folder in vdp_puzzles:
    train = glob(folder + "/train*")
    test  = glob(folder + "/test*")
    name = os.path.basename(folder)
    config = {
        'name' : name,
        'train' : list(train),
        'test' : list(test),
        'sg_config' : sgconfig,
        'fo_config' : foconfig
    }
    json.dump(config, open(f"./config/{name}.json", 'w'))
    

    

## YOLO $\to$ IR

In [24]:
def yolo_to_ir(yolo_folders, output_folder):
    for folder_path in yolo_folders:
        name = os.path.basename(folder_path)
        output_dir =f"{output_folder}/{name}" 
        os.makedirs(output_dir, exist_ok=True)
        fo_models = vdp.utils.get_yolo_fo_models(folder_path)
        for batch in ['test', 'train']:
            batch_path = f"{output_dir}/{batch}"
            os.makedirs(batch_path, exist_ok=True)
            for img_path, fo_model in fo_models:
                if os.path.basename(img_path).split("_")[0] == batch:
                    json_name = os.path.basename(img_path).replace('.jpg', '_model.json')
                    output_json = f"{batch_path}/{json_name}"
                    with open(output_json, 'w') as fp:
                        json.dump(fo_model, fp, indent=2)


yolo_to_ir(glob("./vdp_smt/*/*"), './data/yolo_ir')


## SGG $\to$ IR

In [25]:
def sgg_to_ir(input_dirs, output_dir, box_topk, rel_topk, raw_img_dir):
    
    for input_dir in input_dirs:
        name = os.path.basename(input_dir)
        for batch_dir in ['train', 'test']:
            sg_input_dir = input_dir + f"/{batch_dir}"
            fo_output_dir = output_dir + f"/{name}/{batch_dir}"
            os.makedirs(fo_output_dir, exist_ok=True)
            fo_models = vdp.utils.get_sgg_fo_models(sg_input_dir, box_topk=box_topk, rel_topk=rel_topk, raw_img_dir=raw_img_dir)
            print(f"{name}, {batch_dir}, {len(fo_models)}")
            for img_path, fo_model in fo_models:
                output_json = f"{fo_output_dir}/{os.path.basename(img_path).replace('.jpg', '_model.json')}"
                with open(output_json, 'w') as fp:
                    json.dump(fo_model, fp, indent=2)

        

sgg_to_ir(glob("./data/sg_processed/*"), "./data/sg_ir", 20, 20, "./data/raw")

alldogsonsofas_abc-23d, train, 3
alldogsonsofas_abc-23d, test, 3
kitchen_abc-1.12.1e, train, 3
kitchen_abc-1.12.1e, test, 3
catontv_ab-1.22.1c, train, 2
catontv_ab-1.22.1c, test, 3
topmost_abc-13e, train, 3
topmost_abc-13e, test, 3
2on1_abe-c123, train, 3
2on1_abe-c123, test, 4
sleeping_abc-1.22.2d, train, 3
sleeping_abc-1.22.2d, test, 3
desktop_abc-1.12.13.1e, train, 3
desktop_abc-1.12.13.1e, test, 4
populatedbus_abc-1.12.2d, train, 3
populatedbus_abc-1.12.2d, test, 3
dogherded_abc-1.12.23.1d, train, 3
dogherded_abc-1.12.23.1d, test, 4
umbrella_abc-135d, train, 3
umbrella_abc-135d, test, 4
pplandties_abc-1.22.1e, train, 3
pplandties_abc-1.22.1e, test, 3
parkmeter_abc-234d, train, 3
parkmeter_abc-234d, test, 4
tvon_abc-13d, train, 3
tvon_abc-13d, test, 3
diagonal_ab-14c, train, 2
diagonal_ab-14c, test, 3
pplwearingties_abc-1.12.13.1e, train, 3
pplwearingties_abc-1.12.13.1e, test, 4
samelevel_ab-17c, train, 2
samelevel_ab-17c, test, 3
allcatsonsofas_abc-1.12f, train, 3
allcatsonsofas_ab

## IR $\to$ smt

In [2]:
with open("./data/yolo_ir/2on1_abc-d123/train/train_a_model.json", "r") as f:
    ir = json.load(f)
ir

{'sorts': ['object', 'label'],
 'predicates': {'right': ['object', 'object'],
  'above': ['object', 'object'],
  'left': ['object', 'object'],
  'below': ['object', 'object'],
  'within': ['object', 'object'],
  'has_label': ['object', 'label']},
 'elements': {'object': ['0_sofa', '1_cat'], 'label': ['cat', 'sofa']},
 'interpretation': {'right': [['0_sofa', '1_cat']],
  'above': [['0_sofa', '1_cat']],
  'left': [['1_cat', '0_sofa']],
  'below': [['1_cat', '0_sofa']],
  'within': [['1_cat', '0_sofa']],
  'has_label': [['0_sofa', 'sofa'], ['1_cat', 'cat']]},
 'raw': {'rel_labels': [['0', 'sofa', '1', 'cat', 'right', '1.0'],
   ['0', 'sofa', '1', 'cat', 'above', '1.0'],
   ['1', 'cat', '0', 'sofa', 'left', '1.0'],
   ['1', 'cat', '0', 'sofa', 'below', '1.0'],
   ['1', 'cat', '0', 'sofa', 'within', '1.0']],
  'var_const_map': {'0_sofa': 'sofa', '1_cat': 'cat'},
  'scores': [['0_sofa', '1_cat', '1.0'],
   ['0_sofa', '1_cat', '1.0'],
   ['1_cat', '0_sofa', '1.0'],
   ['1_cat', '0_sofa', '1.0

In [3]:
vdp_sorts = dict()
vdp_constants = dict()

assert('label' in ir['sorts'])
assert('object' in ir['sorts'])
for sort in ir['sorts']:
    vdp_sorts[sort] = FOSort(sort)
    
    
    
for label in ir['elements']['label']:
    if label not in vdp_constants:
        vdp_constants[label] = FOFunction(label, tuple([vdp_sorts['label']]))


unique_labels = list(ir['elements']['label']) # @TODO REPLACE!!!!
unique_labels

['cat', 'sofa']

In [4]:
relation_labels = list(ir['predicates'].keys())

vdp_relations = dict()

for rl in relation_labels:
    sig = tuple([*ir['predicates'][rl], 'bool'])
    vdp_relations[rl] = FOFunction(rl, sig)


vdp_relations


{'right': <vdp.vocabulary.FOFunction at 0x11d67d210>,
 'above': <vdp.vocabulary.FOFunction at 0x11d67d190>,
 'left': <vdp.vocabulary.FOFunction at 0x11d67d1d0>,
 'below': <vdp.vocabulary.FOFunction at 0x11d67d250>,
 'within': <vdp.vocabulary.FOFunction at 0x11d67d290>,
 'has_label': <vdp.vocabulary.FOFunction at 0x11d67d2d0>}

In [5]:
vdp_vocabulary = (set([vdp_sorts['object'], vdp_sorts['label']]), set(list(vdp_constants.values()) + list(vdp_relations.values())))

In [6]:
vdp_vocabulary = (set([vdp_sorts['object'], vdp_sorts['label']]), set(list(vdp_constants.values()) + list(vdp_relations.values())))
vdp_objects = {obj : FOElement(obj, vdp_sorts['object']) for obj, lab in ir['interpretation']['has_label']}
vdp_labels = {lab : FOElement(lab, vdp_sorts['label']) for obj, lab in ir['interpretation']['has_label']}

vdp_elements = {'Object': set(vdp_objects.values()), 'Label': set(vdp_labels.values())}

In [7]:


labelconst_interpretation = {const : vdp_labels[label] for label, const in zip(unique_labels, list(vdp_constants.values()))}
labelconst_interpretation

{<vdp.vocabulary.FOFunction at 0x11d67f590>: <vdp.fomodel.FOElement at 0x11d68db90>,
 <vdp.vocabulary.FOFunction at 0x11d67f650>: <vdp.fomodel.FOElement at 0x11d68dbd0>}

In [8]:
vdp_relations

{'right': <vdp.vocabulary.FOFunction at 0x11d67d210>,
 'above': <vdp.vocabulary.FOFunction at 0x11d67d190>,
 'left': <vdp.vocabulary.FOFunction at 0x11d67d1d0>,
 'below': <vdp.vocabulary.FOFunction at 0x11d67d250>,
 'within': <vdp.vocabulary.FOFunction at 0x11d67d290>,
 'has_label': <vdp.vocabulary.FOFunction at 0x11d67d2d0>}

In [9]:
ir_interpretation = ir['interpretation']
rel_interpretation = dict()
for rel_sym, rel in vdp_relations.items():
    if 'has_label' in rel_sym:
        rel_interpretation[rel] = {(obj, lab) : [obj_sym, lab_sym] in ir_interpretation[rel_sym] for obj_sym, obj in vdp_objects.items() for lab_sym, lab in vdp_labels.items()}
    else:
        rel_interpretation[rel] = {(obj1, obj2) : [obj1_sym, obj2_sym] in ir_interpretation[rel_sym]  for obj1_sym, obj1 in vdp_objects.items() for obj2_sym, obj2 in vdp_objects.items() if obj1_sym != obj2_sym}

        
vdp_interpretation = {**labelconst_interpretation, **rel_interpretation}
model = FOModel(vdp_vocabulary, vdp_elements, vdp_interpretation)


In [10]:
vdp_interpretation = {**labelconst_interpretation, **rel_interpretation}
model = FOModel(vdp_vocabulary, vdp_elements, vdp_interpretation)


In [11]:
model = FOModel(vdp_vocabulary, vdp_elements, vdp_interpretation)


In [12]:
vdp_objects.keys()

dict_keys(['0_sofa', '1_cat'])

In [13]:
# testing
tst = FOElement('x', vdp_sorts['object'])
cat_is_cat = model.interpret(term=[vdp_relations['has_label'], [tst], [vdp_constants['cat']]], interpretation_extension={tst: vdp_objects['1_cat']})
cat_is_sofa = model.interpret(term=[vdp_relations['has_label'], [tst], [vdp_constants['cat']]], interpretation_extension={tst: vdp_objects['0_sofa']})

cat_is_cat, cat_is_sofa

(True, False)

# Undocument code

In [7]:

relations = {}
for label1, pred1, bb1 in img_params:
    for label2, pred2, bb2 in img_params:
        if label1 != label2:
            c1 = centroid(bb1)
            c2 = centroid(bb2)
            relations[(label1, label2)] = relate(c1, c2)
        else:
            relations[(label1, label2)] = ['labelOf']
            
labels, preds, bbs = zip(*img_params)

relations


{('bed', 'bed'): ['labelOf'],
 ('bed', 'sofa'): ['right', 'above'],
 ('bed', 'cat'): ['left', 'above'],
 ('sofa', 'bed'): ['left', 'below'],
 ('sofa', 'sofa'): ['labelOf'],
 ('sofa', 'cat'): ['left', 'above'],
 ('cat', 'bed'): ['right', 'below'],
 ('cat', 'sofa'): ['right', 'below'],
 ('cat', 'cat'): ['labelOf']}

In [8]:
vdpobject = FOSort('Object')
vdplabel = FOSort('Label')
vdpbool = FOSort('Bool')


In [9]:
vdpconstants = dict()
for label in labels:
    if label not in vdpconstants:
        vdpconstants[label] = FOFunction(label, tuple([vdplabel]))
    
    
unique_labels = list(vdpconstants.keys()) # @TODO REPLACE!!!!


In [103]:
vdp_objects

{'0_sofa': <vdp.fomodel.FOElement at 0x11e037410>,
 '1_cat': <vdp.fomodel.FOElement at 0x11e037c10>}

True

In [10]:
# on = FOFunction('On', (vdpobject, vdpobject, vdpbool))
relation_labels = ['left', 'right', 'above', 'below']
vdprelations = dict()

for rl in relation_labels:
    vdprelations[rl] = FOFunction(rl, (vdpobject, vdpobject, vdpbool))


vdprelations['labelOf'] = FOFunction('labelOf', (vdpobject, vdplabel, vdpbool))
vdprelations

{'left': <vdp.vocabulary.FOFunction at 0x10889af10>,
 'right': <vdp.vocabulary.FOFunction at 0x10889a250>,
 'above': <vdp.vocabulary.FOFunction at 0x10889ab90>,
 'below': <vdp.vocabulary.FOFunction at 0x10889a790>,
 'labelOf': <vdp.vocabulary.FOFunction at 0x10889ad10>}

In [11]:

vdpvocabulary = (set([vdpobject, vdplabel]), set(list(vdpconstants.values()) + list(vdprelations.values())))

In [12]:
vdpobjects = {label : FOElement(label + "_object", vdpobject) for label in labels}
vdplabels = {label : FOElement(label + "_label", vdplabel) for label in unique_labels}


vdpelements = {'Object': set(vdpobjects.values()), 'Label': set(vdplabels.values())}

In [13]:
labelconst_interpretation = {const : vdplabels[label] for label, const in zip(unique_labels, list(vdpconstants.values()))}
labelconst_interpretation

{<vdp.vocabulary.FOFunction at 0x10889a650>: <vdp.fomodel.FOElement at 0x111a026d0>,
 <vdp.vocabulary.FOFunction at 0x10889a2d0>: <vdp.fomodel.FOElement at 0x111a02810>,
 <vdp.vocabulary.FOFunction at 0x10889a510>: <vdp.fomodel.FOElement at 0x111a02910>}

In [14]:

    

rel_interpretation = dict()
# for rel_sym, rel in vdprelations.items():
#     if 'labelOf' in rel_sym:
#         rel_interpretation[rel] = {(obj, lab) : rel_sym in relations[(obj_sym, lab_sym)]  for obj_sym, obj in vdpobjects.items() for lab_sym, lab in vdplabels.items()}
#     else:
#         rel_interpretation[rel] = {(obj1, obj2) : rel_sym in relations[(obj1_sym, obj2_sym)]  for obj1_sym, obj1 in vdpobjects.items() for obj2_sym, obj2 in vdpobjects.items() if obj1_sym != obj2_sym}

        
rel_interpretation[vdprelations['labelOf']] = {(obj, lab) : 'labelOf' in relations[(obj_sym, lab_sym)]  for obj_sym, obj in vdpobjects.items() for lab_sym, lab in vdplabels.items()}


rel_interpretation

{<vdp.vocabulary.FOFunction at 0x10889ad10>: {(<vdp.fomodel.FOElement at 0x111a02610>,
   <vdp.fomodel.FOElement at 0x111a026d0>): True,
  (<vdp.fomodel.FOElement at 0x111a02610>,
   <vdp.fomodel.FOElement at 0x111a02810>): False,
  (<vdp.fomodel.FOElement at 0x111a02610>,
   <vdp.fomodel.FOElement at 0x111a02910>): False,
  (<vdp.fomodel.FOElement at 0x111a02710>,
   <vdp.fomodel.FOElement at 0x111a026d0>): False,
  (<vdp.fomodel.FOElement at 0x111a02710>,
   <vdp.fomodel.FOElement at 0x111a02810>): True,
  (<vdp.fomodel.FOElement at 0x111a02710>,
   <vdp.fomodel.FOElement at 0x111a02910>): False,
  (<vdp.fomodel.FOElement at 0x111a02790>,
   <vdp.fomodel.FOElement at 0x111a026d0>): False,
  (<vdp.fomodel.FOElement at 0x111a02790>,
   <vdp.fomodel.FOElement at 0x111a02810>): False,
  (<vdp.fomodel.FOElement at 0x111a02790>,
   <vdp.fomodel.FOElement at 0x111a02910>): True}}

In [15]:
vdpinterpretation = {**labelconst_interpretation, **rel_interpretation}


In [16]:
model = FOModel(vdpvocabulary, vdpelements, vdpinterpretation)


In [17]:
tst = FOElement('x', vdpobject)
model.interpret(term=[vdprelations['labelOf'], [tst], [vdpconstants['cat']]], interpretation_extension={tst: vdpobjects['cat']})

True